In [8]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [37]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython
from math import sqrt

# cdef class Node:
#     cdef public double mass  # Define mass as a double for efficient storage
#     cdef public object old_delta  # 1D array of doubles
#     cdef public object delta      # 1D array of doubles
#     cdef public object position   # 1D array of doubles

#     # cdef inline __cinit__(self, int dim):
#     #     # Initialize each attribute, ensuring arrays are created with the specified dimension
#     #     self.mass = 0.0
#     #     self.old_delta = np.zeros(dim, dtype=np.float64)
#     #     self.delta = np.zeros(dim, dtype=np.float64)
#     #     self.position = np.zeros(dim, dtype=np.float64)


# # This is not in the original java function, but it makes it easier to
# # deal with edges.
# cdef class Edge:
#     cdef public int node1, node2
#     cdef public double weight

# # Repulsion function.  `n1` and `n2` should be nodes.  This will
# # adjust the dx and dy values of `n1` (and optionally `n2`).  It does
# # not return anything.

# @cython.locals(displacement = np.ndarray,
#                distance2 = cython.double,
#                n1_pos = np.ndarray,
#                n2_pos = np.ndarray, 
#                factor = cython.double)
# cdef void linRepulsion(Node n1, Node n2, double coefficient)

# @cython.locals(displacement = np.ndarray,
#                massCenter = np.ndarray,
#                position = np.ndarray,
#                distance2 = cython.double,
#                factor = cython.double)
# cdef void linRepulsion_region(Node n, Region r, double coefficient)


# @cython.locals(displacement = np.ndarray,
#                distance = cython.double, 
#                factor = cython.double)
# cdef void linGravity(Node n, double g)


# @cython.locals(displacement = np.ndarray,
#                factor = cython.double)
# cdef void strongGravity(Node n, double g, double coefficient)

# @cython.locals(displacement = np.ndarray,
#                position1 = np.ndarray,
#                position2 = np.ndarray,
#                factor = cython.double)
# cpdef void linAttraction(Node n1, Node n2, double e, bint distributedAttraction, double coefficient)

# @cython.locals(i = cython.int,
#                j = cython.int,
#                n1 = Node,
#                n2 = Node)
# cpdef void apply_repulsion(list nodes, double coefficient)

# @cython.locals(n = Node)
# cpdef void apply_gravity(list nodes, double gravity, double scalingRatio, bint useStrongGravity)

# @cython.locals(edge = Edge)
# cpdef void apply_attraction(list nodes, list edges, bint distributedAttraction, double coefficient, double edgeWeightInfluence)

# cdef class Region:
#     cdef double mass
#     cdef int dim
#     cdef object massCenter  # 1D array for mass center in given dimensions
#     cdef double size
#     cdef object nodes  # For a list of nodes, we’ll use Python’s dynamic typing
#     cdef list subregions  # List of subregions as a standard Python list

#     # cdef inline __cinit__(self, nodes, int dim):
#     #     self.mass = 0.0
#     #     self.massCenter = np.zeros(dim, dtype=np.float64)
#     #     self.size = 0.0
#     #     self.nodes = nodes
#     #     self.subregions = []

#     @cython.locals(massSum = np.ndarray,
#                    position = np.ndarray,
#                    massCenter = np.ndarray,
#                    n = Node,
#                    distance = cython.double)
#     cdef void updateMassAndGeometry(self)

#     @cython.locals(n = Node,
#                    numSubregions = int,
#                    i = int,
#                    subregions = list,
#                    subregion = Region)
#     cpdef void buildSubRegions(self)


#     @cython.locals(distance = cython.double,
#                    subregion = Region)
#     cdef void applyForce(self, Node n, double theta, double coefficient)

#     @cython.locals(n = Node)
#     cpdef applyForceOnNodes(self, list nodes, double theta, double coefficient)

# @cython.locals(totalSwinging = cython.double,
#                totalEffectiveTraction = cython.double,
#                old_delta = np.ndarray,
#                delta = np.ndarray,
#                position = np.ndarray,
#                n = Node,
#                swinging = cython.double,
#                totalSwinging = cython.double,
#                totalEffectiveTraction = cython.double,
#                estimatedOptimalJitterTolerance = cython.double,
#                minJT = cython.double,
#                maxJT = cython.double,
#                jt = cython.double,
#                minSpeedEfficiency = cython.double,
#                targetSpeed = cython.double,
#                maxRise = cython.double,
#                factor = cython.double,
#                values = dict)
# cpdef dict adjustSpeedAndApplyForces(list nodes, double speed, double speedEfficiency, double jitterTolerance)


# This will substitute for the nLayout object
class Node:
    def __init__(self, dim):
        self.mass = 0.0
        # self.old_dx = 0.0
        # self.old_dy = 0.0
        # self.dx = 0.0
        # self.dy = 0.0
        # self.x = 0.0
        # self.y = 0.0
        self.old_delta = np.zeros(dim, dtype=np.double)
        self.delta = np.zeros(dim, dtype=np.double)
        self.position = np.zeros(dim, dtype=np.double)
        assert len(self.old_delta.shape) == 1, "old_delta is not a 1D array"


# This is not in the original java code, but it makes it easier to deal with edges
class Edge:
    def __init__(self):
        self.node1 = -1
        self.node2 = -1
        self.weight = 0.0


# Here are some functions from ForceFactory.java
# =============================================

# Repulsion function.  `n1` and `n2` should be nodes.  This will
# adjust the dx and dy values of `n1`  `n2`
def linRepulsion(n1, n2, coefficient=0):
    # xDist = n1.x - n2.x
    # yDist = n1.y - n2.y
    # convert to numpy array
    n1_pos = n1.position
    n2_pos = n2.position
    displacement = n1_pos - n2_pos
    distance2 = np.sum(displacement ** 2)  # Distance squared
    # distance2 = xDist * xDist + yDist * yDist  # Distance squared

    if distance2 > 0:
        factor = coefficient * n1.mass * n2.mass / distance2
        # n1.dx += xDist * factor
        # n1.dy += yDist * factor
        # n2.dx -= xDist * factor
        # n2.dy -= yDist * factor
        n1.delta += displacement * factor
        n2.delta -= displacement * factor


# Repulsion function. 'n' is node and 'r' is region
def linRepulsion_region(n, r, coefficient=0):
    # xDist = n.x - r.massCenterX
    # yDist = n.y - r.massCenterY
    # distance2 = xDist * xDist + yDist * yDist
    position = n.position
    massCenter = r.massCenter
    displacement = position - massCenter
    distance2 = np.sum(displacement ** 2)

    if distance2 > 0:
        factor = coefficient * n.mass * r.mass / distance2
        # n.dx += xDist * factor
        # n.dy += yDist * factor
        n.delta += displacement * factor


# Gravity repulsion function.  For some reason, gravity was included
# within the linRepulsion function in the original gephi java code,
# which doesn't make any sense (considering a. gravity is unrelated to
# nodes repelling each other, and b. gravity is actually an
# attraction)
def linGravity(n, g):
    # xDist = n.x
    # yDist = n.y
    # distance = sqrt(xDist * xDist + yDist * yDist)
    displacement = n.position
    distance = np.sqrt(np.sum(displacement ** 2))

    if distance > 0:
        factor = n.mass * g / distance
        # n.dx -= xDist * factor
        # n.dy -= yDist * factor
        n.delta -= displacement * factor


# Strong gravity force function. `n` should be a node, and `g`
# should be a constant by which to apply the force.
def strongGravity(n, g, coefficient=0):
    # xDist = n.x
    # yDist = n.y
    # if xDist != 0 and yDist != 0:
    #     factor = coefficient * n.mass * g
    #     n.dx -= xDist * factor
    #     n.dy -= yDist * factor
    displacement = n.position
    if np.all(displacement != 0):
        factor = coefficient * n.mass * g
        n.delta -= displacement * factor



# Attraction function.  `n1` and `n2` should be nodes.  This will
# adjust the dx and dy values of `n1` and `n2`.  It does
# not return anything.
def linAttraction(n1, n2, e, distributedAttraction, coefficient=0):
    # xDist = n1.x - n2.x
    # yDist = n1.y - n2.y
    # displacement = n1.position - n2.position
    position1 = n1.position
    position2 = n2.position
    displacement = position1 - position2
    if not distributedAttraction:
        factor = -coefficient * e
    else:
        factor = -coefficient * e / n1.mass
    # n1.dx += xDist * factor
    # n1.dy += yDist * factor
    # n2.dx -= xDist * factor
    # n2.dy -= yDist * factor
    n1.delta += displacement * factor
    n2.delta -= displacement * factor


# The following functions iterate through the nodes or edges and apply
# the forces directly to the node objects.  These iterations are here
# instead of the main file because Python is slow with loops.
def apply_repulsion(nodes, coefficient):
    i = 0
    for n1 in nodes:
        j = i
        for n2 in nodes:
            if j == 0:
                break
            linRepulsion(n1, n2, coefficient)
            j -= 1
        i += 1


def apply_gravity(nodes, gravity, scalingRatio, useStrongGravity=False):
    if not useStrongGravity:
        for n in nodes:
            linGravity(n, gravity)
    else:
        for n in nodes:
            strongGravity(n, gravity, scalingRatio)


def apply_attraction(nodes, edges, distributedAttraction, coefficient, edgeWeightInfluence):
    # Optimization, since usually edgeWeightInfluence is 0 or 1, and pow is slow
    if edgeWeightInfluence == 0:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], 1, distributedAttraction, coefficient)
    elif edgeWeightInfluence == 1:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], edge.weight, distributedAttraction, coefficient)
    else:
        for edge in edges:
            linAttraction(nodes[edge.node1], nodes[edge.node2], pow(edge.weight, edgeWeightInfluence),
                          distributedAttraction, coefficient)


# For Barnes Hut Optimization
class Region:
    def __init__(self, nodes, dim):
        self.mass = 0.0
        # self.massCenterX = 0.0
        # self.massCenterY = 0.0
        self.dim = dim
        self.massCenter = np.zeros(dim)
        self.size = 0.0
        self.nodes = nodes
        self.subregions = []
        self.updateMassAndGeometry()

    def updateMassAndGeometry(self):
        if len(self.nodes) > 1:
            self.mass = 0
            # massSumX = 0
            # massSumY = 0
            massSum = np.zeros(self.dim)
            for n in self.nodes:
                self.mass += n.mass
                # massSumX += n.x * n.mass
                # massSumY += n.y * n.mass
                position = n.position
                # print(np.asarray(n.position).shape)
                massSum += position * n.mass
            # self.massCenterX = massSumX / self.mass
            # self.massCenterY = massSumY / self.mass
            self.massCenter = massSum / self.mass

            self.size = 0.0
            for n in self.nodes:
                # distance = sqrt((n.x - self.massCenterX) ** 2 + (n.y - self.massCenterY) ** 2)
                position = n.position
                massCenter = self.massCenter
                distance = np.sqrt(np.sum((position - massCenter) ** 2))
                self.size = max(self.size, 2 * distance)

    def buildSubRegions(self):
        if len(self.nodes) > 1:
            # topleftNodes = []
            # bottomleftNodes = []
            # toprightNodes = []
            # bottomrightNodes = []

            # 2 ^ dim subregions
            numSubregions = int(2 ** len(self.massCenter))
            subregions = [[] for _ in range(numSubregions)]

            # Optimization: The distribution of self.nodes into 
            # subregions now requires only one for loop. Removed 
            # topNodes and bottomNodes arrays: memory space saving.
            for n in self.nodes:
                # partition into hypercubes centered at self.massCenter
                subregionIndex = 0
                for i in range(len(self.massCenter)):
                    if n.position[i] > self.massCenter[i]:
                        subregionIndex += int(2 ** i)
                subregions[subregionIndex].append(n)
                # if n.x < self.massCenterX:
                #     if n.y < self.massCenterY:
                #         bottomleftNodes.append(n)
                #     else:
                #         topleftNodes.append(n)
                # else:
                #     if n.y < self.massCenterY:
                #         bottomrightNodes.append(n)
                #     else:
                #         toprightNodes.append(n)      
            for subregionNodes in subregions:
                if len(subregionNodes) > 0:
                    if len(subregionNodes) < len(self.nodes):
                        subregion = Region(subregionNodes, dim=len(self.massCenter))
                        self.subregions.append(subregion)
                    else:
                        for n in subregionNodes:
                            subregion = Region([n])
                            self.subregions.append(subregion)

            # if len(topleftNodes) > 0:
            #     if len(topleftNodes) < len(self.nodes):
            #         subregion = Region(topleftNodes)
            #         self.subregions.append(subregion)
            #     else:
            #         for n in topleftNodes:
            #             subregion = Region([n])
            #             self.subregions.append(subregion)

            # if len(bottomleftNodes) > 0:
            #     if len(bottomleftNodes) < len(self.nodes):
            #         subregion = Region(bottomleftNodes)
            #         self.subregions.append(subregion)
            #     else:
            #         for n in bottomleftNodes:
            #             subregion = Region([n])
            #             self.subregions.append(subregion)

            # if len(toprightNodes) > 0:
            #     if len(toprightNodes) < len(self.nodes):
            #         subregion = Region(toprightNodes)
            #         self.subregions.append(subregion)
            #     else:
            #         for n in toprightNodes:
            #             subregion = Region([n])
            #             self.subregions.append(subregion)

            # if len(bottomrightNodes) > 0:
            #     if len(bottomrightNodes) < len(self.nodes):
            #         subregion = Region(bottomrightNodes)
            #         self.subregions.append(subregion)
            #     else:
            #         for n in bottomrightNodes:
            #             subregion = Region([n])
            #             self.subregions.append(subregion)

            for subregion in self.subregions:
                subregion.buildSubRegions()

    def applyForce(self, n, theta, coefficient=0):
        if len(self.nodes) < 2:
            linRepulsion(n, self.nodes[0], coefficient)
        else:
            # distance = sqrt((n.x - self.massCenterX) ** 2 + (n.y - self.massCenterY) ** 2)
            position = n.position
            massCenter = self.massCenter
            distance = np.sqrt(np.sum((position - massCenter) ** 2))
            # distance = np.sqrt(np.sum((n.position - self.massCenter) ** 2))
            if distance * theta > self.size:
                linRepulsion_region(n, self, coefficient)
            else:
                for subregion in self.subregions:
                    subregion.applyForce(n, theta, coefficient)

    def applyForceOnNodes(self, nodes, theta, coefficient=0):
        for n in nodes:
            self.applyForce(n, theta, coefficient)


# Adjust speed and apply forces step
def adjustSpeedAndApplyForces(nodes, speed, speedEfficiency, jitterTolerance):
    # Auto adjust speed.
    totalSwinging = 0.0  # How much irregular movement
    totalEffectiveTraction = 0.0  # How much useful movement
    for n in nodes:
        # swinging = sqrt((n.old_dx - n.dx) * (n.old_dx - n.dx) + (n.old_dy - n.dy) * (n.old_dy - n.dy))
        old_delta = n.old_delta
        delta = n.delta
        # swinging = np.sqrt(np.sum((n.old_delta - n.delta) ** 2))
        swinging = np.sqrt(np.sum((old_delta - delta) ** 2))
        totalSwinging += n.mass * swinging
        # totalEffectiveTraction += .5 * n.mass * sqrt(
            # (n.old_dx + n.dx) * (n.old_dx + n.dx) + (n.old_dy + n.dy) * (n.old_dy + n.dy))
        # totalEffectiveTraction += .5 * n.mass * np.sqrt(np.sum((n.old_delta + n.delta) ** 2))
        totalEffectiveTraction += .5 * n.mass * np.sqrt(np.sum((old_delta + delta) ** 2))
    # Optimize jitter tolerance.  The 'right' jitter tolerance for
    # this network. Bigger networks need more tolerance. Denser
    # networks need less tolerance. Totally empiric.
    estimatedOptimalJitterTolerance = .05 * sqrt(len(nodes))
    minJT = sqrt(estimatedOptimalJitterTolerance)
    maxJT = 10
    jt = jitterTolerance * max(minJT,
                               min(maxJT, estimatedOptimalJitterTolerance * totalEffectiveTraction / (
                                   len(nodes) * len(nodes))))

    minSpeedEfficiency = 0.05

    # Protective against erratic behavior
    if totalEffectiveTraction and totalSwinging / totalEffectiveTraction > 2.0:
        if speedEfficiency > minSpeedEfficiency:
            speedEfficiency *= .5
        jt = max(jt, jitterTolerance)

    if totalSwinging == 0:
        targetSpeed = float('inf')
    else:
        targetSpeed = jt * speedEfficiency * totalEffectiveTraction / totalSwinging

    if totalSwinging > jt * totalEffectiveTraction:
        if speedEfficiency > minSpeedEfficiency:
            speedEfficiency *= .7
    elif speed < 1000:
        speedEfficiency *= 1.3

    # But the speed shoudn't rise too much too quickly, since it would
    # make the convergence drop dramatically.
    maxRise = .5
    speed = speed + min(targetSpeed - speed, maxRise * speed)

    # Apply forces.
    #
    # Need to add a case if adjustSizes ("prevent overlap") is
    # implemented.
    for n in nodes:
        # swinging = n.mass * sqrt((n.old_dx - n.dx) * (n.old_dx - n.dx) + (n.old_dy - n.dy) * (n.old_dy - n.dy))
        # swinging = n.mass * np.sqrt(np.sum((n.old_delta - n.delta) ** 2))
        old_delta = n.old_delta
        delta = n.delta
        swinging = n.mass * np.sqrt(np.sum((old_delta - delta) ** 2))
        factor = speed / (1.0 + sqrt(speed * swinging))
        # n.x = n.x + (n.dx * factor)
        # n.y = n.y + (n.dy * factor)

        # n.position = n.position + (n.delta * factor)
        position = n.position
        # print("before: ", np.asarray(n.position).shape)
        n.position = position + (delta * factor)
        # print("after: ", np.asarray(n.position).shape)
        # print()
        # print()

    values = {}
    values['speed'] = speed
    values['speedEfficiency'] = speedEfficiency

    return values

Content of stderr:
In file included from /home/tristan/envs/isorc/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /home/tristan/envs/isorc/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tristan/envs/isorc/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tristan/.cache/ipython/cython/_cython_magic_6f476e6309fc6c069a14a213637da7afefd0b6e5.c:1274:
/home/tristan/envs/isorc/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~